<a target="_parent" href="https://colab.research.google.com/github/gretelai/gretel-blueprints/blob/main/docs/notebooks/demo/navigator/text-to-code/navigator-data-designer-sdk-text-to-python.ipynb">
  <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
</a>

# 🎨 Navigator Data Designer SDK: Text-to-Python

This notebook demonstrates how to use the Gretel Navigator SDK to create a synthetic data generation pipeline for Python code examples. We'll build a system that generates Python code based on natural language instructions, with varying complexity levels and industry focuses.

In [ ]:
%%capture
%pip install -U gretel_client 

In [ ]:
from gretel_client.navigator import DataDesigner

## 📘 Setting Up the Data Designer

First, we'll initialize the Data Designer with appropriate system instructions.

In [ ]:
# Initialize the Data Designer
data_designer = DataDesigner(
    api_key="prompt", 
    model_suite="apache-2.0",  # Use apache-2.0 or llama-3.x based on your licensing needs
    endpoint="https://api.gretel.cloud",
    special_system_instructions="""\
You are an expert at writing, analyzing, and editing Python code. You know what
high-quality, clean, efficient, and maintainable Python code looks like. You
excel at transforming natural language into Python, as well as Python back into
natural language. Your job is to assist the user with their Python-related tasks.
"""
) 

## 🌱 Define Categorical Seed Columns

We'll set up our seed columns for industry sectors, code complexity, and instruction types. These will help generate diverse and relevant code examples.

In [ ]:
# Add industry sector categories
data_designer.add_categorical_seed_column(
    name="industry_sector",
    description="The industry sector for the code example",
    values=["Healthcare", "Finance", "Technology"],
    subcategories=[
        {
            "name": "topic",
            "values": {
                "Healthcare": [
                    "Electronic Health Records (EHR) Systems",
                    "Telemedicine Platforms", 
                    "AI-Powered Diagnostic Tools"
                ],
                "Finance": [
                    "Fraud Detection Software",
                    "Automated Trading Systems",
                    "Personal Finance Apps"
                ],
                "Technology": [
                    "Cloud Computing Platforms",
                    "Artificial Intelligence and Machine Learning Platforms",
                    "DevOps and CI/CD Tools"
                ]
            }
        }
    ]
)

# Add code complexity and concepts
data_designer.add_categorical_seed_column(
    name="code_complexity",
    description="The complexity level of the code",
    values=["Beginner", "Intermediate", "Advanced"],
    subcategories=[
        {
            "name": "code_concept",
            "values": {
                "Beginner": [
                    "Variables",
                    "Data Types",
                    "Functions",
                    "Loops",
                    "Classes"
                ],
                "Intermediate": [
                    "List Comprehensions",
                    "Object-oriented programming",
                    "Lambda Functions",
                    "Web frameworks",
                    "Pandas"
                ],
                "Advanced": [
                    "Multithreading",
                    "Context Managers",
                    "Generators"
                ]
            }
        }
    ]
)

# Add instruction phrases
data_designer.add_categorical_seed_column(
    name="instruction_phrase",
    description="Starting phrase for the code instruction",
    values=[
        "Write a function that",
        "Create a class that",
        "Implement a script",
        "Can you create a function",
        "Develop a module that"
    ]
)

## ✨ Define Generated Data Columns

Now we'll set up the columns that will be generated by the LLMs, including the instruction and code implementation.

In [ ]:
# Generate instruction for the code
data_designer.add_generated_data_column(
    name="instruction",
    generation_prompt="""\
Generate an instruction to create Python code that solves a specific problem. 
Each instruction should begin with one of the following phrases: {instruction_phrase}.

Important Guidelines:
* Industry Relevance: Ensure the instruction pertains to the {industry_sector} sector and {topic} topic.
* Code Complexity: Tailor the instruction to the {code_complexity} level. Utilize relevant {code_concept} where appropriate to match the complexity level.
* Clarity and Specificity: Make the problem statement clear and unambiguous. Provide sufficient context to understand the requirements without being overly verbose.
* Response Formatting: Do not include any markers such as ### Response ### in the instruction.
"""
)

# Generate the Python code
data_designer.add_generated_data_column(
    name="code_implementation",
    llm_type="code",
    data_config={"type": "code", "params": {"syntax": "python"}},
    generation_prompt="""\
Write Python code for the following instruction:
Instruction: {instruction}

Important Guidelines:
* Code Quality: Your code should be clean, complete, self-contained and accurate.
* Code Validity: Please ensure that your python code is executable and does not contain any errors.
* Packages: Remember to import any necessary libraries, and to use all libraries you import.
* Complexity & Concepts: The code should be written at a {code_complexity} level, making use of concepts such as {code_concept}.
"""
)

## 🔍 Add Validation and Evaluation

Let's add post-processing steps to validate the generated code and evaluate the text-to-Python conversion.

In [ ]:
data_designer.add_validator(
    validator="code",
    code_lang="python",
    code_columns=["code_implementation"]
)

data_designer.add_evaluator(
    eval_type="text_to_python",  
    instruction_column_name="instruction",    
    response_column_name="code_implementation"
)

## 👀 Generate Preview Dataset

Let's generate a preview to see some data.

In [ ]:
preview = data_designer.generate_dataset_preview()

## 🔎 Easily inspect individual records

- Run the cell below to display individual records for inspection.

- Run the cell multiple times to cycle through the 10 preview records.

- Alternatively, you can pass the `index` argument to `display_sample_record` to display a specific record.

In [ ]:
preview.display_sample_record()

## 🚀 Generate Full Dataset

If you're satisfied with the preview, you can generate a larger dataset using a batch workflow.

In [ ]:
# Submit batch job
batch_job = data_designer.submit_batch_workflow(num_records=100)
df = batch_job.fetch_dataset(wait_for_completion=True)
print("\nGenerated dataset shape:", df.shape)

# Download evaluation report
path = batch_job.download_evaluation_report(wait_for_completion=True)